<a href="https://colab.research.google.com/github/srirakshareddy/airbnb/blob/main/airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###Neural Network

import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import scipy.stats

# Read the LA_Lising.csv
df = pd.read_csv('LA_Listings.csv', encoding='ISO-8859-1')
df.head()
# print("hii")

# Read the NY_Listings.csv
df2 = pd.read_csv('NY_Listings.csv', encoding='ISO-8859-1')
df2.head()

# Read the airbnb_ratings_new.csv
df3 = pd.read_csv('airbnb_ratings_new.csv', encoding='ISO-8859-1')
pd.set_option('display.max_columns', None)
df3.head()

filter_df = df3[df3['Country'] == 'United States']

filter_df.head()

df_combined = df.append(df2)
df_final = df_combined.append(filter_df)
excel_file = pd.ExcelWriter('Final Dataset.xlsx')
df_final.to_excel(excel_file, index=False)

excel_file.save()

df_final = pd.read_excel('/content/drive/MyDrive/Pattern Recognization/Final Dataset.xlsx')

df_final.describe()

# Commented out IPython magic to ensure Python compatibility.
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
#Scikit-learn for implemeting LinearRegression from a existing algorithm.
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample

# Common imports
import numpy as np
import pandas as pd

from IPython.display import clear_output

# To plot pretty figures
# %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

def cost_compute(X, y, theta):
    return 1/(2*y.size)*np.sum(np.square(X.dot(theta)-y))

from sklearn.model_selection import train_test_split
df_final, test = train_test_split(df_final, test_size=0.3, random_state=43)

# Drop any rows with null value
df_final.dropna(axis=0, how='any', inplace=True)
test.dropna(axis=0, how='any', inplace=True)

column_names = ['Host total listings count', 'longitude', 'Accommodates', 'Bathrooms', 'Bedrooms', 'Minimum nights', 'Maximum nights', 'Availability 365', 'Number of reviews',
                'Review Scores Accuracy', 'Review Scores Cleanliness', 'Review Scores Checkin', 'Review Scores Communication', 'Review Scores Location', 'Review Scores Value', 'Reviews per month']
X_train = df_final[column_names]
y_train = df_final['Price']

X_test = test[column_names]
y_test = test['Price']

scaler = MinMaxScaler()
input_train_scaled = scaler.fit_transform(X_train)
output_train_scaled = scaler.fit_transform(y_train.values.reshape(-1,1))
input_test_scaled = scaler.fit_transform(X_test)
output_test_scaled = scaler.fit_transform(y_test.values.reshape(-1,1))

input_train_scaled.shape

output_test_scaled.shape

def activation_relu(z): # takes a numpy array as input and returns activated array
    relu = np.maximum(0,z)
    return relu

def parameter_initialization(size_layer): #takes a list of the layer sizes as input and returns initialized parameters
    param = {}
    for i in range(1, len(size_layer)):
        param['W' + str(i)] = np.random.randn(size_layer[i], size_layer[i-1])*0.01
        param['B' + str(i)] = np.random.randn(size_layer[i],1)*0.01
    return param

def forward_pass(X_train, param): #takes input training features and parameters as input and returns a dictionary containining the numpy arrays of activations of all layer
    layer = len(param)//2
    value = {}
    for i in range(1, layer+1):
        if i==1:
            value['Z' + str(i)] = np.dot(param['W' + str(i)], X_train) + param['B' + str(i)]
            value['A' + str(i)] = activation_relu(value['Z' + str(i)])
        else:
            value['Z' + str(i)] = np.dot(param['W' + str(i)], value['A' + str(i-1)]) + param['B' + str(i)]
            if i==layer:
                value['A' + str(i)] = value['Z' + str(i)]
            else:
                value['A' + str(i)] = activation_relu(value['Z' + str(i)])
    return value

def compute_cost(value, Y_train): #takes true value and dictionary having activations of all layer as input and returns cost
    layer = len(value)//2
    Y_pred = value['A' + str(layer)]
    cost = 1/(2*len(Y_train)) * np.sum(np.square(Y_pred - Y_train))
    return cost

def back_propagation(param, value, X_train, Y_train): #takes parameters, activations, training set as input and returns gradients wrt parameters
    layer = len(param)//2
    m = len(Y_train)
    grads = {}
    for i in range(layer,0,-1):
        if i==layer:
            dA = 1/m * (value['A' + str(i)] - Y_train)
            dZ = dA
        else:
            dA = np.dot(param['W' + str(i+1)].T, dZ)
            dZ = np.multiply(dA, np.where(value['A' + str(i)]>=0, 1, 0))
        if i==1:
            grads['W' + str(i)] = 1/m * np.dot(dZ, X_train.T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        else:
            grads['W' + str(i)] = 1/m * np.dot(dZ,value['A' + str(i-1)].T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return grads

def param_update(param, grads, lr): #takes parameters, gradients and learning rate as input and returns updated parameters
    layer = len(param)//2
    params_updated = {}
    for i in range(1,layer+1):
        params_updated['W' + str(i)] = param['W' + str(i)] - lr * grads['W' + str(i)]
        params_updated['B' + str(i)] = param['B' + str(i)] - lr * grads['B' + str(i)]
    return params_updated

def NN(X_train, Y_train, size_layer, n_iters, lr): #trains the NN
    param = parameter_initialization(size_layer)
    for i in range(n_iters):
        value = forward_pass(X_train.T, param)
        cost = compute_cost(value, Y_train.T)
        grads = back_propagation(param, value,X_train.T, Y_train.T)
        param = param_update(param, grads, lr)
        print('Cost at iteration ' + str(i+1) + ' = ' + str(cost) + '\n')
    return param

def accuracy(X_train, X_test, Y_train, Y_test, param): #compute accuracy on test and training data given learnt parameters
    values_train = forward_pass(X_train.T, param)
    values_test = forward_pass(X_test.T, param)
    train_acc = np.sqrt(mean_squared_error(Y_train, values_train['A' + str(len(size_layer)-1)].T))
    test_acc = np.sqrt(mean_squared_error(Y_test, values_test['A' + str(len(size_layer)-1)].T))
    return train_acc, test_acc

def predict(X, param):  #predict on new array X given learnt parameters
    value = forward_pass(X.T, param)
    predictions = value['A' + str(len(value)//2)].T
    return predictions

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def activation_relu(z):
    a = np.maximum(0,z)
    return a

def parameter_initialization(size_layer):
    param = {}
    for i in range(1, len(size_layer)):
        param['W' + str(i)] = np.random.randn(size_layer[i], size_layer[i-1])*0.01
        param['B' + str(i)] = np.random.randn(size_layer[i],1)*0.01
    return param

def forward_pass(X_train, param):
    layer = len(param)//2
    value = {}
    for i in range(1, layer+1):
        if i==1:
            value['Z' + str(i)] = np.dot(param['W' + str(i)], X_train) + param['B' + str(i)]
            value['A' + str(i)] = activation_relu(value['Z' + str(i)])
        else:
            value['Z' + str(i)] = np.dot(param['W' + str(i)], value['A' + str(i-1)]) + param['B' + str(i)]
            if i==layer:
                value['A' + str(i)] = value['Z' + str(i)]
            else:
                value['A' + str(i)] = activation_relu(value['Z' + str(i)])
    return value

def compute_cost(value, Y_train):
    layer = len(value)//2
    Y_pred = value['A' + str(layer)]
    cost = 1/(2*len(Y_train)) * np.sum(np.square(Y_pred - Y_train))
    return cost

def back_propagation(param, value, X_train, Y_train):
    layer = len(param)//2
    m = len(Y_train)
    grads = {}
    for i in range(layer,0,-1):
        if i==layer:
            dA = 1/m * (value['A' + str(i)] - Y_train)
            dZ = dA
        else:
            dA = np.dot(param['W' + str(i+1)].T, dZ)
            dZ = np.multiply(dA, np.where(value['A' + str(i)]>=0, 1, 0))
        if i==1:
            grads['W' + str(i)] = 1/m * np.dot(dZ, X_train.T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        else:
            grads['W' + str(i)] = 1/m * np.dot(dZ,value['A' + str(i-1)].T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return grads

def param_update(param, grads, lr):
    layer = len(param)//2
    params_updated = {}
    for i in range(1,layer+1):
        params_updated['W' + str(i)] = param['W' + str(i)] - lr * grads['W' + str(i)]
        params_updated['B' + str(i)] = param['B' + str(i)] - lr * grads['B' + str(i)]
    return params_updated

def NN(X_train, Y_train, size_layer, n_iters, lr):
    param = parameter_initialization(size_layer)
    for i in range(n_iters):
        value = forward_pass(X_train.T, param)
        cost = compute_cost(value, Y_train.T)
        grads = back_propagation(param, value,X_train.T, Y_train.T)
        param = param_update(param, grads, lr)
        print('Cost at iteration ' + str(i+1) + ' = ' + str(cost) + '\n')
    return param

def accuracy(X_train, X_test, Y_train, Y_test, param):
    values_train = forward_pass(X_train.T, param)
    values_test = forward_pass(X_test.T, param)
    train_acc = np.sqrt(mean_squared_error(Y_train, values_train['A' + str(len(size_layer)-1)].T))
    test_acc = np.sqrt(mean_squared_error(Y_test, values_test['A' + str(len(size_layer)-1)].T))
    return train_acc, test_acc

def predict(X, param):
    value = forward_pass(X.T, param)
    predictions = value['A' + str(len(value)//2)].T
    return predictions

size_layer = [16, 5, 5, 1]                                                       #set layer sizes, do not change the size of the first and last layer 
n_iters = 200                                                                  #set number of iterations over the training set(also known as epochs in batch gradient descent context)
lr = 0.000001                                                              #set learning rate for gradient descent
param = NN(input_train_scaled, output_train_scaled, size_layer, n_iters, lr)           #train the NN
train_acc, test_acc = accuracy(input_train_scaled, input_test_scaled, output_train_scaled, output_test_scaled, param)  #get training and test accuracy
print('RMSE Training Data = ' + str(train_acc))
print('RMSE Test Data = ' + str(test_acc))

from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

regr = MLPRegressor(random_state=1, max_iter=500).fit(input_train_scaled, output_train_scaled)

regr.score(input_test_scaled, output_test_scaled)

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

DT = DecisionTreeRegressor(random_state = 16, criterion = 'mse', max_depth = None, splitter = 'best')
DT.fit(input_train_scaled, output_train_scaled)

pred = DT.predict(input_test_scaled)
r2_score(output_test_scaled, pred)

from sklearn.svm import SVR
from sklearn.metrics import r2_score
regressor=SVR(kernel='rbf',gamma='auto')

regressor.fit(input_train_scaled, output_train_scaled)
test_pre = regressor.predict(input_test_scaled)
train_pre = regressor.predict(input_train_scaled)

prediction = regressor.predict(input_test_scaled)
mse = mean_squared_error(output_test_scaled, prediction)
rmse = mse**.5
r2 = r2_score(output_test_scaled, prediction)
print(mse)
print(rmse)
print(r2)

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 5, random_state = 16).fit(input_train_scaled, output_train_scaled)

prediction = rf.predict(input_test_scaled)
mse = mean_squared_error(output_test_scaled, prediction)
rmse = mse**.5
r2 = r2_score(output_test_scaled, prediction)
print(mse)
print(rmse)
print(r2)

